In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import binarize
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier


In [3]:
df = pd.read_csv('data/small_dataset.csv', index_col=0)
df

,index,Opcodes,Label
0,0,PUSH1 PUSH1 MSTORE PUSH1 CALLDATASIZE LT PUSH2...,0
1,1,PUSH1 PUSH1 MSTORE PUSH1 CALLDATASIZE LT PUSH2...,0
2,2,PUSH1 PUSH1 MSTORE PUSH1 CALLDATASIZE LT PUSH2...,0
3,3,PUSH1 PUSH1 MSTORE PUSH1 CALLDATASIZE LT PUSH2...,0
4,4,PUSH1 PUSH1 MSTORE PUSH1 CALLDATASIZE LT PUSH2...,0
...,...,...,...
2087,2130,PUSH1 PUSH1 MSTORE CALLVALUE DUP1 ISZERO PUSH2...,1
2088,2131,PUSH1 PUSH1 MSTORE CALLVALUE DUP1 ISZERO PUSH2...,1
2089,2132,PUSH1 PUSH1 MSTORE CALLVALUE DUP1 ISZERO PUSH2...,1
2090,2134,PUSH1 PUSH1 MSTORE PUSH1 CALLDATASIZE LT PUSH2...,1


Vectorizing with tf-idf and bagging with 4 ngrams

In [4]:
# transform opcodes to tf-idf vectors
vectorizer = TfidfVectorizer(ngram_range=(1, 4))

X = vectorizer.fit_transform(df['Opcodes'])

X_train, X_test, y_train, y_test = train_test_split(X, df['Label'], test_size=0.2, random_state=1)

Using SMOTE and StratifiedCV to deal with imbalanced data |
Comparing Logistic Reggression, Random Forset and XGboost

In [5]:
from imblearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score


# Define the cross-validation procedure
cv = StratifiedKFold(n_splits=2)

classifiers = [LogisticRegression(), RandomForestClassifier(), XGBClassifier()]
model_names = ["Logistic Regression", "Random Forest", "XGBoost"]
results = {"Model": [], "Mean Accuracy": [], "Std Accuracy": [], "Mean Precision": [], "Mean Recall": [], "Mean F1 Score": []}


scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

smote = SMOTE()

for clf, model_name in zip(classifiers, model_names):
    
    # Create pipeline
    pipeline = Pipeline([
        ('sampling', smote),
        ('classification', clf)
    ])
    
    scores = cross_validate(pipeline, X, df['Label'], scoring=scoring, cv=cv, n_jobs=-1, return_train_score=False)
    
    results["Model"].append(model_name)
    results["Mean Accuracy"].append(scores['test_accuracy'].mean())
    results["Std Accuracy"].append(scores['test_accuracy'].std())
    results["Mean Precision"].append(scores['test_precision'].mean())
    results["Mean Recall"].append(scores['test_recall'].mean())
    results["Mean F1 Score"].append(scores['test_f1_score'].mean())

results_df = pd.DataFrame(results)
print(results_df)


                 Model  Mean Accuracy  Std Accuracy  Mean Precision  \
0  Logistic Regression       0.628107      0.179732        0.091133   
1        Random Forest       0.910612      0.057839        0.446015   
2              XGBoost       0.903920      0.059751        0.392148   

   Mean Recall  Mean F1 Score  
0     0.684783       0.155928  
1     0.576087       0.429233  
2     0.576087       0.406070  


After evaluating the diffrent models we will now train them on all the data and export the weights

In [7]:
# Fitting classifiers to the whole data
logistic_model = LogisticRegression()
logistic_model.fit(X, df['Label'])

rf_model = RandomForestClassifier()
rf_model.fit(X, df['Label'])

xgb_model = XGBClassifier()
xgb_model.fit(X, df['Label'])


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [8]:
import pickle

# Assuming that logistic_model, rf_model, xgb_model, and tfidf are already defined

with open('weights/logistic_model.pkl', 'wb') as f:
    pickle.dump(classifiers[0], f)

with open('weights/rf_model.pkl', 'wb') as f:
    pickle.dump(classifiers[1], f)

with open('weights/xgb_model.pkl', 'wb') as f:
    pickle.dump(classifiers[2], f)
    
with open('weights/tfidf.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)